In [1]:
import pandas as pd
import numpy as np
from map_functions import *

df = pd.read_csv('data/locations (no_lat-lng).csv')
df.tail()

,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des
9,"Belfast, Ireland",NaN,NaN,"Dublin, Ireland",NaN,NaN
10,"Wolf's Castle, UK",NaN,NaN,"Dublin, Ireland",NaN,NaN
11,"London, UK",NaN,NaN,"Dublin, Ireland",NaN,NaN
12,"Amsterdam, Netherlands",NaN,NaN,"Dublin, Ireland",NaN,NaN
13,"Copenhagen, Denmark",NaN,NaN,"Dublin, Ireland",NaN,NaN


In [2]:
# Add columns for origin and destination payloads (for Google API) 
payload_pre = 'https://maps.googleapis.com/maps/api/geocode/json?address='
df['Payload_Orig'] = df['Name_Orig'].str.replace(', ','+').str.replace(' ','+')
df['Payload_Des'] = df['Name_Des'].str.replace(', ','+').str.replace(' ','+')
df['Payload_Orig'] = payload_pre + df['Payload_Orig']
df['Payload_Des'] = payload_pre + df['Payload_Des']
df['Distance (nm)'] = 0
df['Distance (mi)'] = 0
df['Distance (km)'] = 0
df.tail()

,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des,Payload_Orig,Payload_Des,Distance (nm),Distance (mi),Distance (km)
9,"Belfast, Ireland",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
10,"Wolf's Castle, UK",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
11,"London, UK",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
12,"Amsterdam, Netherlands",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0
13,"Copenhagen, Denmark",NaN,NaN,"Dublin, Ireland",NaN,NaN,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,0,0,0


I round the decimal degrees and distance measurements for readability. Pick what to round decimal degrees to based on the accuracy required. For our purposes, 3 decimal places should suffice. 

This table [from Wikipedia](https://en.wikipedia.org/wiki/Decimal_degrees#Precision) gives the precision of decimal degrees. 

| Decimal Degrees	| Identifiable	| Distance At Equator | 
|------|------|------|
|1	| country or large region	| 111.32 km |
|0.1	| large city or district	| 11.132 km |
|0.01	| town or village	| 1.1132 km |
|0.001	| neighborhood, street	| 111.32 m |
|0.0001	| individual street, land parcel	| 11.132 m |
|0.00001	| individual trees	| 1.1132 m |
|0.000001	| individual humans	| 111.32 mm |
|0.0000001	| practical limit of commercial surveying	| 11.132 mm |
|0.00000001	| specialized surveying (e.g. tectonic plate mapping)	| 1.1132 mm |

In [3]:
# Find latitude and longitude of each location, and distance between
round_coord = 3
round_dist = 1

print('Completed...')
for row in range(0,df.shape[0]):
    try: 
        # Read payloads to pass to Google API
        payload_orig = df.iloc[row,6]
        payload_des = df.iloc[row,7]
        
        # Request and write latitude and longitude
        lat_orig, lng_orig = get_lat_lng(payload_orig)
        lat_des, lng_des = get_lat_lng(payload_des)
        df.ix[row,'Lat_Orig'] = round(lat_orig,round_coord)
        df.ix[row,'Lng_Orig'] = round(lng_orig,round_coord)
        df.ix[row,'Lat_Des'] = round(lat_des,round_coord)
        df.ix[row,'Lng_Des'] = round(lng_des,round_coord)
        
        # Calculate and write distance
        nm, mi, km = get_distance(lat_orig, lng_orig, lat_des, lng_des)
        df.ix[row,'Distance (nm)']  = round(nm,round_dist)
        df.ix[row,'Distance (mi)']  = round(mi,round_dist)
        df.ix[row,'Distance (km)'] = round(km,round_dist)
        
        print(' - '+ str(round(nm,round_dist))+' nm '+df.iloc[row,0]+' to '+df.iloc[row,3])
    
    except:
        print('\tSKIPPING: '+df.iloc[row,0]+' to '+df.iloc[row,3])
        print('\t'+payload_orig)
        print('\t'+payload_des)

df.tail()

Completed...
 - 80.9 nm Dublin, Ireland to Isle of Man, UK
 - 117.5 nm Dublin, Ireland to Liverpool, UK
 - 75.9 nm Dublin, Ireland to Belfast, Ireland
 - 99.0 nm Dublin, Ireland to Wolf's Castle, UK
 - 250.9 nm Dublin, Ireland to London, UK
 - 409.5 nm Dublin, Ireland to Amsterdam, Netherlands
 - 670.9 nm Dublin, Ireland to Copenhagen, Denmark
 - 80.9 nm Isle of Man, UK to Dublin, Ireland
 - 117.5 nm Liverpool, UK to Dublin, Ireland
 - 75.9 nm Belfast, Ireland to Dublin, Ireland
 - 99.0 nm Wolf's Castle, UK to Dublin, Ireland
 - 250.9 nm London, UK to Dublin, Ireland
 - 409.5 nm Amsterdam, Netherlands to Dublin, Ireland
 - 670.9 nm Copenhagen, Denmark to Dublin, Ireland


,Name_Orig,Lat_Orig,Lng_Orig,Name_Des,Lat_Des,Lng_Des,Payload_Orig,Payload_Des,Distance (nm),Distance (mi),Distance (km)
9,"Belfast, Ireland",54.597,-5.930,"Dublin, Ireland",53.35,-6.26,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,75.9,87.3,140.5
10,"Wolf's Castle, UK",51.902,-4.971,"Dublin, Ireland",53.35,-6.26,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,99.0,113.9,183.3
11,"London, UK",51.507,-0.128,"Dublin, Ireland",53.35,-6.26,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,250.9,288.7,464.6
12,"Amsterdam, Netherlands",52.370,4.895,"Dublin, Ireland",53.35,-6.26,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,409.5,471.3,758.4
13,"Copenhagen, Denmark",55.676,12.568,"Dublin, Ireland",53.35,-6.26,https://maps.googleapis.com/maps/api/geocode/j...,https://maps.googleapis.com/maps/api/geocode/j...,670.9,772.1,1242.5


In [4]:
# Save df to csv
header = ['Name_Orig','Lat_Orig','Lng_Orig','Name_Des','Lat_Des','Lng_Des',
          'Distance (nm)','Distance (mi)','Distance (km)']
df.to_csv('data/locations.csv', columns=header, index=False)